アルゴリズムの検討

- 上がりそうな銘柄を買いポジションで持つための銘柄選別

In [ ]:
from pathlib import Path
import csv
from typing import List
import datetime

import numpy as np
import matplotlib.pyplot as plt

import stock

In [ ]:
code_list_csv = stock.constants.DATA_DIR / "margin_trade_target_list.csv"
daily_data_dir = Path.home() / "remote/gdrive/stock/data/daily"
one_minute_data_dir = Path.home() / "remote/gdrive/stock/data/one_minute"

assert code_list_csv.exists()
assert daily_data_dir.exists()
assert one_minute_data_dir.exists()

with open(code_list_csv) as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    codes = [row[0] for row in csv_reader]

In [ ]:
def get_daily_data(code, data_dir=daily_data_dir) -> List:
    csvs = sorted(data_dir.glob(f"*{code}.csv"))
    if len(csvs) == 0:
        stock.logger.warning(f"No csv files of `{code}` in the directory.")
        return []
    daily_data_csv = csvs[0]
    assert daily_data_csv.exists()
    with open(daily_data_csv) as f:
        csv_reader = csv.reader(f)
        header = next(csv_reader)
        data = [
            [
                datetime.datetime.strptime(row[0], "%Y/%m/%d").timestamp() + 60 * 60 * 9,  # utcで0時になるように変換
                float(row[2]),
                float(row[3]),
                float(row[4]),
                float(row[5]),
                float(row[6])
            ]
            for row in csv_reader
        ]
    return np.array(data)

In [ ]:
data = get_daily_data(codes[0])

In [ ]:
window_sizes = [5, 20]

fig, ax = stock.visualize.boxplot(data[:, 1], data[:, 2], data[:, 3], data[:, 4])

for window_size in window_sizes:
    ma_start = stock.chart.trend.moving_average(data[:, 1], window_size=window_size)
    ma_end = stock.chart.trend.moving_average(data[:, 4], window_size=window_size)
    ma_diff = ma_end[1:] - ma_end[:-1]

    ax.plot(np.arange(1, len(ma_end) + 1), ma_end)

In [ ]:
raw_diff = data[:, 4] - data[:, 1]
print("Number of start < end  : {} / {} ({:.2f} %)".format(
    (raw_diff > 0).sum(), len(raw_diff), (raw_diff > 0).sum() / len(raw_diff) * 100
))